In [2]:
import pandas as pd
import getpass
from py2neo import Graph, Node, Relationship
from py2neo import GraphService
import neo4jupyter
print("Import success. ")

Import success. 


In [3]:
# Manually enter host URL for neo4j database. Localhost with exposed port or base ICICLE url will go here.
host = getpass.getpass(prompt='Host: ', stream=None)

# Input for password - generally "pass" or "password", but can sometimes be custom.
# For ICICLE hosted stuff on TAPIS, this will be a property of the pod. 
password = getpass.getpass(prompt='Password: ', stream=None) 

# Creating the graph object to operate on the Neo4j database.
graph = Graph(host, auth=("neo4j", password))

Host: ········
Password: ········


In [4]:
# Testing connection with a simple query. Should display all of the components (limited to 5) in a table.
query = "MATCH(n:COMPONENT) RETURN n LIMIT 5"
graph.run(query).to_data_frame()

n
0  {'repositories': 'Code Repositories', 'last_up...
1  {'repositories': 'https://github.com/orgs/tapi...
2  {'repositories': 'https://github.com/tapis-pro...
3  {'last_update': '10/4/2022', 'name': 'Event En...
4  {'repositories': 'https://github.com/tapis-pro...

In [ ]:
# Create all components from scratch
componentsFromScratch = """
LOAD CSV FROM "https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/ICICLE%20CI%20Components%20Catalog%20-%20ICCLE%20Software%20Components%20Inventory%20-%20Oct%202022.csv" AS line
CREATE (:COMPONENT {name: line[0], description: line[1], standalone: line[4], testing_process: line[9], releases: line[10], repositories: line[11], req_and_usage: line[12], status: line[13], last_update: line[14]})
"""
# graph.run(componentsFromScratch)

In [21]:
# Create all contacts from scratch
contactsFromScratch = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/ICICLE%20CI%20Components%20Catalog%20-%20ICCLE%20Software%20Components%20Inventory%20-%20Oct%202022.csv' AS row
CREATE (c:CONTACT {name: row.`Person of Contact`})
"""
deleteDuplicates = """ 
MATCH (c:CONTACT)
WITH c.name AS name, collect(c) AS nodes 
WHERE size(nodes) >  1
FOREACH (c in tail(nodes) | DETACH DELETE c)
"""

#graph.run(contactsFromScratch)
#graph.run(deleteDuplicates)

# Note: there is a bug here, since when there are multiple contacts the "duplicates" don't get deleted


(No data)

In [24]:
# Create all contacts from scratch
ownerFromScratch = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/ICICLE%20CI%20Components%20Catalog%20-%20ICCLE%20Software%20Components%20Inventory%20-%20Oct%202022.csv' AS row
CREATE (o:OWNER {name: row.`Owning Group (AI4CI, CI4AI, etc)`})
"""
deleteDuplicates = """ 
MATCH (o:OWNER)
WITH o.name AS name, collect(o) AS nodes 
WHERE size(nodes) >  1
FOREACH (o in tail(nodes) | DETACH DELETE o)
"""

graph.run(ownerFromScratch)
graph.run(deleteDuplicates)

# Note: there is a bug here, since when there are multiple owners the "duplicates" don't get deleted


(No data)

In [22]:
# Create All Use Cases from scratch 

useCasesFromScratch = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/ICICLE%20CI%20Components%20Catalog%20-%20ICCLE%20Software%20Components%20Inventory%20-%20Oct%202022.csv' AS row
CREATE (u:USE_INSPIRED {name: row.`Which Use-inspired Science Cases will be using it in the future?`})
"""
graph.run(useCasesFromScratch)

useCasesFromScratch2 = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/sdsc-hpc-students/REHS2022/main/ICICLE%20CI%20Components%20Catalog%20-%20ICCLE%20Software%20Components%20Inventory%20-%20Oct%202022.csv' AS row
CREATE (u:USE_INSPIRED {name: row.`Which Use-inspired science case is using it now?`})
"""
graph.run(useCasesFromScratch2)

deleteDuplicates = """ 
MATCH (u:USE_INSPIRED)
WITH u.name AS name, collect(u) AS nodes 
WHERE size(nodes) >  1
FOREACH (u in tail(nodes) | DETACH DELETE u)
"""
graph.run(deleteDuplicates)
# Note: there is a bug here, since when there are multiple use cases the "duplicates" don't get deleted


(No data)

In [9]:
# This code block handles creating dependency relationships between existing component nodes

name = "\"" + str(input("Enter component name: ")) + "\""
script = "MATCH (n1:COMPONENT {name: " + name + "})"
dependencies = int(input("Enter number of dependencies: "))
for i in range(dependencies):
    dependency = "\"" + str(input("Enter dependency name: ")) + "\""
    script += "MATCH (n" + str(i) + ":COMPONENT {name: " + dependency + "})"
    script += "CREATE (n1) -[:DEPENDS_ON]-> (n" + str(i) + ")"

graph.run(script)

Enter component name: Pods Service (previously KG API)
Enter number of dependencies: 1
Enter dependency name: Base ICICLE Tapis


(No data)

In [ ]:
# In progress - each of these relationships is for use case
# Each must have existing/future component


name = "\"" + str(input("Enter component name: ")) + "\""
script = "MATCH (n1:COMPONENT {name: " + name + "})"
useCases = int(input("Enter number of Use-Inspired Cases: "))
for i in range(useCases):
    useCase = "\"" + str(input("Enter Use Case name: ")) + "\""
    script += "MATCH (n" + str(i) + ":COMPONENT {name: " + dependency + "})"
    script += "CREATE (n1) -[:DEPENDS_ON]-> (n" + str(i) + ")"

graph.run(script)

In [ ]:
"""
name = str(input("Enter component name: "))
description = str(input("Enter component description: "))
contact = str(input("Enter contact for component: "))
owningGroup = str(input("Enter component owning group for component: "))
standalone = str(input("Is the component standalong from Tapis? Enter \"Yes\" or \"No\": "))
dependsOn = str(input("Enter dependencies on other components: "))
"""
